In [83]:
from __future__ import division, print_function
import sympy as sm
from sympy import trigsimp
from sympy import latex
import sympy.physics.mechanics as me
import mpmath as mp
import symbtools as st
import numpy as np
import symbtools.modeltools as mt
import symbtools.noncommutativetools as nct
from scipy.integrate import odeint

import pycartan as pc
from sympy.solvers import solve

from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)

#definingymbolic variables
q = st.symb_vector('q:2')
qdot = st.time_deriv(q, q)
qdd = st.time_deriv(qdot, qdot)
st.make_global(q, qdot)

f = sm.symbols('f')
m = sm.symbols('m:2')
J = sm.symbols('J:2')
l = sm.symbols('l')
a = sm.symbols('a')
d = sm.symbols('d')
g, t = sm.symbols('g t')

# defining reference frames and points
In_frame = me.ReferenceFrame('In_frame')  #reference frame with velocity 0
O = me.Point('O')
O.set_vel(In_frame, 0)

# Mass center of the cart
C0 = me.Point('C0')
C0.set_pos(O, q[0] * In_frame.x)
C0.set_vel(In_frame, qdot[0] * In_frame.x)

#cart frame
cart_frame = In_frame.orientnew('L0', 'Axis', [q[0], In_frame.x])
cart_frame.set_ang_vel(In_frame, 0)

#endulum frame
pen_frame = In_frame.orientnew('L1', 'Axis', [q[1], In_frame.z])
pen_frame.set_ang_vel(In_frame, qdot[1] * In_frame.z)
#mass center of pendulum
s = C0.locatenew('a', a * pen_frame.x)
s.v2pt_theory(C0, In_frame, pen_frame)


# finding velocities in inertial reference frame --> use them to find kinetic energy
v0 = C0.vel(In_frame)
v1 = s.vel(In_frame)
T = 1 / 2 * m[0] * v0.dot(v0) + 1 / 2 * m[1] * v1.dot(
    v1) + 1 / 2 * J[1] * qdot[1]**2

# potential energy
h = s.pos_from(O).dot(In_frame.y)
#V = 1 / 2 * m[0] * g * h - 1/2 * d * qdot[1]**2
V = 1 / 2 * m[0] * g * h
tau = -d * qdot[1]

params = [m[0], m[1], J[0], J[1], a, d, g]
st.make_global(params)

mod = mt.generate_symbolic_model(T, V, q, [f, tau])
mod.eqns.simplify()


In [84]:
mod.calc_state_eq(simplify=True)

In [85]:
xx=mod.xx
fx=mod.f
G=mod.g
#gx, =st.col_split(G)

In [86]:
fx

Matrix([
[                                                                                                                                      qdot0],
[                                                                                                                                      qdot1],
[1.0*a*m1*(-qdot1**2*(J1 + a**2*m1)*cos(q1) + (0.5*a*g*m0*cos(q1) + 1.0*d*qdot1)*sin(q1))/(a**2*m1**2*sin(q1)**2 - (J1 + a**2*m1)*(m0 + m1))],
[   1.0*(-a**2*m1**2*qdot1**2*sin(2*q1)/2 + (m0 + m1)*(0.5*a*g*m0*cos(q1) + 1.0*d*qdot1))/(a**2*m1**2*sin(q1)**2 - (J1 + a**2*m1)*(m0 + m1))]])

In [87]:
xx

Matrix([
[   q0],
[   q1],
[qdot0],
[qdot1]])

In [88]:
G

Matrix([
[                                                                         0],
[                                                                         0],
[-(1.0*J1 + 1.0*a**2*m1)/(a**2*m1**2*sin(q1)**2 - (J1 + a**2*m1)*(m0 + m1))],
[      -1.0*a*m1*sin(q1)/(a**2*m1**2*sin(q1)**2 - (J1 + a**2*m1)*(m0 + m1))]])

In [89]:
equilib_point=sm.Matrix([0, sm.pi/3, 0, 0])
#fx.subz(xx, equilib_point)

In [90]:
parameter_values = [(g, 9.81), (a, 0.2), (d, 0.00715), (m[0], 3.34), (m[1], 0.8512), (J[0], 0), (J[1], 0.01980)]
#replm = list(map(lambda a,b :(a,b),xx, equilib_point)) + parameter_values
replm = parameter_values

In [100]:
#tt=np.arange(0,10, 1e-3)
frames_per_sec= 60
final_time= 50
tt=np.linspace(0.0, final_time, final_time*frames_per_sec)
xx0= st.to_np(equilib_point).ravel()

In [101]:
simulation=st.SimulationModel(mod.f, mod.g , mod.xx, model_parameters=parameter_values)

In [102]:
#help(st.SimulationModel)

In [103]:
rhs1= simulation.create_simfunction()

In [104]:
#help(st.SimulationModel)

In [105]:
sol= odeint(rhs1, xx0, tt)

In [106]:
sol

array([[ 0.00000000e+00,  1.04719755e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.64424705e-04,  1.04251695e+00, -1.97122829e-02, -5.61905256e-01],
       [-6.56712956e-04,  1.02842569e+00, -3.93180391e-02, -1.13018021e+00],
       ...,
       [ 2.22564891e-02, -1.61874721e+00, -6.37867095e-03,  1.57219634e-01],
       [ 2.21317931e-02, -1.61567396e+00, -8.55490778e-03,  2.10828739e-01],
       [ 2.19721741e-02, -1.61174062e+00, -1.05613570e-02,  2.60232060e-01]])